In [1]:
# Install the packages on the instance
!pip install nltk
!pip install pandas
!pip install sklearn

# Make the packages available on the instance
import nltk
import pandas
import sklearn

# Download the components we need to normalize and featurize the text
nltk.download( "averaged_perceptron_tagger" )
nltk.download( "omw-1.4" )
nltk.download( "punkt" )
nltk.download( "stopwords" )
nltk.download( "wordnet" )

# Give colab access to gdrive
from google.colab import drive
drive.mount('/content/gdrive')

# Import other things
import random
import re
import shutil

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Mounted at /content/gdrive


In [4]:
# Find the news category on the drive
import os
stash_dir_path = "/content/gdrive/MyDrive/EAI6010/"
os.listdir( stash_dir_path )

['FaceMask.zip',
 'CatandDog.zip',
 'BrainTumor.zip',
 'Fruits.zip',
 'fruit.model',
 'tweet-sentiment-extraction.zip',
 'module_3_fine_tuned_language_model_lstm_01',
 'module_3_tweet_sentiment_classifier']

In [5]:
# Unzip the data set
!unzip "/content/gdrive/MyDrive/EAI6010/tweet-sentiment-extraction.zip"
print( "\n\n" )
os.listdir( )

Archive:  /content/gdrive/MyDrive/EAI6010/tweet-sentiment-extraction.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               





['.config',
 'gdrive',
 'train.csv',
 'sample_submission.csv',
 'test.csv',
 'sample_data']

In [7]:
import pandas as pd
import numpy as np

In [8]:
# load the 'train.csv'
df = pd.read_csv('train.csv')
df

# We have the text ID, text, selected_text, and sentiment categories.

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive


In [9]:
df.isna().sum()

textID           0
text             1
selected_text    1
sentiment        0
dtype: int64

In [10]:
# Fill NaN values in 'selected_text' with empty strings and ensure all entries are strings
df['selected_text'] = df['selected_text'].fillna('').astype(str)

In [11]:
# Let's follow the convention of a folder train-test, a folder per class, and
# file for an individual record

# Extract categories and texts
category_subset = df['sentiment'].unique()

# Reset the root directory if we need to -- we check and delete so this code is repeatable
train_dir_name = "texts_by_category_train"
test_dir_name = "texts_by_category_test"
print( f"setting up '{train_dir_name}' and '{test_dir_name}'" )
if os.path.exists( train_dir_name ):
  shutil.rmtree( train_dir_name )
os.mkdir( train_dir_name )
if os.path.exists( test_dir_name ):
  shutil.rmtree( test_dir_name )
os.mkdir( test_dir_name )

# Initialize the counts for each of the subset
category_to_count = {category: 0 for category in category_subset}

# Create a file for each record and put it in a subfolder named for the category
count = 0
total_words = 0 # we will create an embedding so knowing this allows us to get an idea of its dimension
train_fraction = 0.75 # roughly how much of the data will we train on?
random.seed( 42 ) # seed the random number generator
num_train = 0
num_test = 0

for index, row in df.iterrows():
    category = row['sentiment']
    document_text = row['selected_text']

    # Update the processed count and show the progress if we should
    count += 1
    if count % 10000 == 0:
        print(f"Creating file for record {int(count / 1000)}k")

    # Pick the root directory
    if random.random( ) <= train_fraction:
      root_dir_name = train_dir_name
      num_train = num_train + 1
    else:
      root_dir_name = test_dir_name
      num_test = num_test + 1

    # Setup the directory if it doesn't exist
    dir_path = os.path.join(root_dir_name, category)
    if not os.path.exists(dir_path):
        os.mkdir(dir_path)

    # Save the document text to file
    category_to_count[category] += 1
    filename = f"{category}-{category_to_count[category]}.txt"
    file_path = os.path.join(dir_path, filename)

    with open(file_path, "w") as f:
        f.write(document_text)

    # Save the word length
    total_words += len(document_text.split(" "))

# Verify the directories
print(f"\nWorking directory:\n{os.listdir()}")
print( f"\ntraining data directory:\n{os.listdir( train_dir_name )}" )
for category in category_subset[:2]:  # just checking first two categories for brevity
    category_dir_path = os.path.join(train_dir_name, category)
    print(f"\n'{category_dir_path}'':\n{os.listdir(category_dir_path)}")

# Verify a document from the second category
category_dir_path = os.path.join(root_dir_name, category_subset[1])
sample_file_path = os.path.join(category_dir_path, f"{category_subset[1]}-1.txt")
with open(sample_file_path, 'r') as f:
    document_content = f.read()
    print(f"\n{category_subset[1]} document content:\n{document_content}")

# Calculate the average document length so we can intelligently set the sequence length
avg_document_words = round(total_words / count)
print(f"\nAverage number of words in a document = {avg_document_words}")

# Print the train-test document counts
print( f"\n{num_train} train documents, {num_test} test documents" )

setting up 'texts_by_category_train' and 'texts_by_category_test'
Creating file for record 10k
Creating file for record 20k

Working directory:
['.config', 'texts_by_category_test', 'gdrive', 'train.csv', 'sample_submission.csv', 'test.csv', 'texts_by_category_train', 'sample_data']

training data directory:
['positive', 'negative', 'neutral']

'texts_by_category_train/neutral'':
['neutral-10200.txt', 'neutral-4093.txt', 'neutral-5670.txt', 'neutral-1949.txt', 'neutral-5839.txt', 'neutral-5748.txt', 'neutral-1819.txt', 'neutral-2419.txt', 'neutral-444.txt', 'neutral-2315.txt', 'neutral-9343.txt', 'neutral-3014.txt', 'neutral-4534.txt', 'neutral-8994.txt', 'neutral-969.txt', 'neutral-7339.txt', 'neutral-7774.txt', 'neutral-48.txt', 'neutral-7902.txt', 'neutral-3432.txt', 'neutral-8375.txt', 'neutral-2917.txt', 'neutral-2307.txt', 'neutral-2618.txt', 'neutral-1467.txt', 'neutral-7309.txt', 'neutral-4484.txt', 'neutral-9759.txt', 'neutral-10812.txt', 'neutral-7873.txt', 'neutral-3696.txt'

In [12]:
# Before we can featurize the documents, we need to create some functions that
# we will use to get the text to a normalized state. Here we are trying to
# increase the likelihood that the features have significant samples to be
# useful.


# The functions need some pieces from NLTK
lemmatizer = nltk.stem.WordNetLemmatizer( )
stop_words = nltk.corpus.stopwords.words( )


# Converts NLTK tagged tokens to WordNet.
def _convert_nltk_to_wordnet( nltk_tagged_tokens ):
    wordnet_tagged_tokens = list( )
    for token, nltk_tag in nltk_tagged_tokens:
        wordnet_tag = None
        if nltk_tag.startswith('J'):
            wordnet_tag = nltk.corpus.wordnet.ADJ
        elif nltk_tag.startswith('V'):
            wordnet_tag = nltk.corpus.wordnet.VERB
        elif nltk_tag.startswith('N'):
            wordnet_tag = nltk.corpus.wordnet.NOUN
        elif nltk_tag.startswith('R'):
            wordnet_tag = nltk.corpus.wordnet.ADV
        wordnet_tagged_tokens.append( ( token, wordnet_tag ) )
    return wordnet_tagged_tokens


# Normalize the supplied text.
def _normalize( text ):

    #   Lemmatize and remove stop words
    tokens = nltk.word_tokenize( text.lower( ) )
    nltk_tagged_tokens = nltk.pos_tag( tokens )
    wordnet_tagged_tokens = _convert_nltk_to_wordnet( nltk_tagged_tokens )
    lemmatized_tokens = list( )
    for token, tag in wordnet_tagged_tokens:
        if token not in stop_words: # keep only non-stop words
            if tag is not None: # lemmatize if we know the parts of speech (POS)
                lemmatized_token = lemmatizer.lemmatize(token, tag)
                lemmatized_tokens.append(lemmatized_token)
            else: # we need POS to lemmatize - we will drop but we could just use
                #lemmatized_tokens.append( token )
                pass

    #   Return the normalized text
    normalized_text = ' '.join( lemmatized_tokens )
    return normalized_text


# Let's demonstrate what is going on
document = "I really like to drink coffees in the morning with two pieces of toast with jam."
normalized_document = _normalize( document )
print( f"document: {document}\nnormalized_document: {normalized_document}" )


document: I really like to drink coffees in the morning with two pieces of toast with jam.
normalized_document: drink coffee morning piece toast jam


In [13]:
# Since we are going to do this more than once, let's define a function to
# load the documents and labels in a directory.
def _load_documents_and_labels( root_dir_path ):
  documents = []
  labels = []
  for label in os.listdir( root_dir_path ):
    dir_path = os.path.join( root_dir_path, label )
    for filename in os.listdir( dir_path ):
      file_path = os.path.join( dir_path, filename )
      with open( file_path, 'r' ) as f:
        document = f.readlines( )
        document = ''.join( document )
      documents.append( document )
      labels.append( label )
  return documents, labels

# Load the training documents
train_documents, train_labels = _load_documents_and_labels( train_dir_name )
print( f"num train documents: {len( train_documents )}" )
print( f"num train labels: {len( train_labels )}" )
print( f"\n{train_labels[ 1 ]}:\n{train_documents[ 1 ]}\n")

# Normalize the training documents
normalized_train_documents = list( )
for i in range( len( train_documents ) ):
  if ( ( i + 1 ) % 1000 ) == 0:
    print( f"normalizing {round((i+1)/1000)}k of {round(len(train_documents)/1000)}k training documents" )
  document = train_documents[ i ]
  normalized_document = _normalize( document )
  normalized_train_documents.append( normalized_document )
print( f"\nnum normalized train documents: {len( normalized_train_documents )}" )
print( f"\n{train_labels[ 1 ]}:\n{normalized_train_documents[ 1 ]}\n")


num train documents: 20601
num train labels: 20601

positive:
..impressive

normalizing 1k of 21k training documents
normalizing 2k of 21k training documents
normalizing 3k of 21k training documents
normalizing 4k of 21k training documents
normalizing 5k of 21k training documents
normalizing 6k of 21k training documents
normalizing 7k of 21k training documents
normalizing 8k of 21k training documents
normalizing 9k of 21k training documents
normalizing 10k of 21k training documents
normalizing 11k of 21k training documents
normalizing 12k of 21k training documents
normalizing 13k of 21k training documents
normalizing 14k of 21k training documents
normalizing 15k of 21k training documents
normalizing 16k of 21k training documents
normalizing 17k of 21k training documents
normalizing 18k of 21k training documents
normalizing 19k of 21k training documents
normalizing 20k of 21k training documents

num normalized train documents: 20601

positive:
.. impressive



In [14]:
# We can train a TF-IDF vectorizer on the training documents
tfidf_vectorizer = sklearn.feature_extraction.text.TfidfVectorizer( max_features = 500 )
tfidf_vectorizer.fit( normalized_train_documents )
x_column_names = tfidf_vectorizer.get_feature_names_out( )
print( f"featurized words: {x_column_names}" )

featurized words: ['absolutely' 'account' 'add' 'afraid' 'afternoon' 'ago' 'agree' 'amaze'
 'amazing' 'annoy' 'answer' 'anymore' 'apparently' 'appreciate' 'asleep'
 'aw' 'awake' 'away' 'awesome' 'awful' 'aww' 'awww' 'baby' 'back' 'bad'
 'band' 'bank' 'bday' 'beach' 'beat' 'beautiful' 'bed' 'beer' 'believe'
 'bet' 'big' 'birthday' 'bit' 'black' 'bless' 'block' 'blog' 'blue' 'boo'
 'book' 'bore' 'boring' 'boy' 'break' 'breakfast' 'bring' 'brother' 'bum'
 'burn' 'bus' 'busy' 'buy' 'bye' 'cake' 'call' 'camera' 'cancel' 'car'
 'card' 'catch' 'chance' 'change' 'chat' 'check' 'cheer' 'chill'
 'chocolate' 'city' 'class' 'clean' 'close' 'co' 'coffee' 'cold' 'college'
 'com' 'come' 'comment' 'company' 'completely' 'computer' 'concert'
 'congrats' 'congratulation' 'contact' 'cool' 'count' 'couple' 'course'
 'crazy' 'cream' 'cry' 'cut' 'cute' 'cuz' 'dad' 'dance' 'date' 'day'
 'dead' 'deal' 'dear' 'decide' 'definitely' 'delicious' 'die' 'different'
 'dinner' 'disappointed' 'dm' 'doctor' 'dream' 'dr

In [15]:
result = tfidf_vectorizer.transform( [ 'elite college demand big change fraternity' ] ).toarray( )[ 0 ].tolist( )
result.sort( reverse=True )
print( f"{result}" )

[0.6411018003274667, 0.5618629995279957, 0.5227795437641806, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [16]:
# Let's create a function to get x-y data frames for documents
def _create_x_and_y_data_frames( normalized_documents, labels, vectorizer ):
  x_rows = list( )
  y_rows = list( )
  for i in range( len( labels ) ):
    if ( ( i + 1 ) % 1000 ) == 0:
      print( f"collecting {round((i+1)/1000)}k of {round(len(train_documents)/1000)}k of rows" )
    x_row = vectorizer.transform( [ normalized_documents[ i ] ] )
    x_row = x_row.toarray( )[ 0 ].tolist( )
    x_rows.append( x_row )
    y_rows.append( [ labels[ i ] ] )
  x_df = pandas.DataFrame( x_rows, columns=vectorizer.get_feature_names_out( ) )
  y_df = pandas.DataFrame( y_rows, columns=[ "label" ] )
  return x_df, y_df

# Lets train and evaluate a classifier
print( "creating x and y data frames" )
x_train, y_train = _create_x_and_y_data_frames( normalized_train_documents, train_labels, tfidf_vectorizer )
print( "training a classifier" )
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier( class_weight="balanced" )
classifier.fit( x_train, y_train )
print( "evaluating the classifier on the training data\n" )
y_train_estimates = classifier.predict( x_train )
report = sklearn.metrics.classification_report( y_train, y_train_estimates )
print( report )

creating x and y data frames
collecting 1k of 21k of rows
collecting 2k of 21k of rows
collecting 3k of 21k of rows
collecting 4k of 21k of rows
collecting 5k of 21k of rows
collecting 6k of 21k of rows
collecting 7k of 21k of rows
collecting 8k of 21k of rows
collecting 9k of 21k of rows
collecting 10k of 21k of rows
collecting 11k of 21k of rows
collecting 12k of 21k of rows
collecting 13k of 21k of rows
collecting 14k of 21k of rows
collecting 15k of 21k of rows
collecting 16k of 21k of rows
collecting 17k of 21k of rows
collecting 18k of 21k of rows
collecting 19k of 21k of rows
collecting 20k of 21k of rows
training a classifier


<ipython-input-16-00adbb2e0e18>:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit( x_train, y_train )


evaluating the classifier on the training data

              precision    recall  f1-score   support

    negative       0.60      0.92      0.73      5816
     neutral       0.91      0.78      0.84      8323
    positive       0.90      0.64      0.75      6462

    accuracy                           0.78     20601
   macro avg       0.81      0.78      0.77     20601
weighted avg       0.82      0.78      0.78     20601



In [17]:
# Let's repeat but for the test data.

# Load the test documents
test_documents, test_labels = _load_documents_and_labels( test_dir_name )
print( f"num test documents: {len( test_documents )}" )
print( f"num test labels: {len( test_labels )}" )
print( f"\n{test_labels[ 0 ]}:\n{test_documents[ 0 ]}\n")

# Normalize the test documents
normalized_test_documents = list( )
for i in range( len( test_documents ) ):
  if ( ( i + 1 ) % 1000 ) == 0:
    print( f"normalizing {round((i+1)/1000)}k of {round(len(test_documents)/1000)}k test documents" )
  document = test_documents[ i ]
  normalized_document = _normalize( document )
  normalized_test_documents.append( normalized_document )
print( f"\nnum normalized test documents: {len( normalized_test_documents )}" )
print( f"\n{test_labels[ 0 ]}:\n{normalized_test_documents[ 0 ]}\n")

# Let's see how we do on the test documents
print( "creating x and y data frames" )
x_test, y_test = _create_x_and_y_data_frames( normalized_test_documents, test_labels, tfidf_vectorizer )
print( "evaluating the classifier on the test data\n" )
y_test_estimates = classifier.predict( x_test )
report = sklearn.metrics.classification_report( y_test, y_test_estimates )
print( report )


num test documents: 6880
num test labels: 6880

positive:
glad

normalizing 1k of 7k test documents
normalizing 2k of 7k test documents
normalizing 3k of 7k test documents
normalizing 4k of 7k test documents
normalizing 5k of 7k test documents
normalizing 6k of 7k test documents

num normalized test documents: 6880

positive:
glad

creating x and y data frames
collecting 1k of 21k of rows
collecting 2k of 21k of rows
collecting 3k of 21k of rows
collecting 4k of 21k of rows
collecting 5k of 21k of rows
collecting 6k of 21k of rows
evaluating the classifier on the test data

              precision    recall  f1-score   support

    negative       0.55      0.80      0.65      1965
     neutral       0.78      0.73      0.75      2795
    positive       0.83      0.56      0.67      2120

    accuracy                           0.70      6880
   macro avg       0.72      0.70      0.69      6880
weighted avg       0.73      0.70      0.70      6880



In [18]:
# Let's calculate a simple test accuracy for comparison
correct_count = 0
for i in range( len( y_test ) ):
  if y_test_estimates[ i ] == y_test.iat[ i, 0 ]:
    correct_count = correct_count + 1
accuracy = round( 100 * ( correct_count / len( y_test ) ) )
print( f'test accuracy: {accuracy}%' )

test accuracy: 70%
